In [18]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

PLAYLIST_ID = os.getenv("PLAYLIST_ID")
API_KEY = os.getenv("YOUTUBE_API_KEY")


def get_playlist_videos(api_key, playlist_id):
    url = "https://www.googleapis.com/youtube/v3/playlistItems"
    videos = []
    next_page_token = ""

    while True:
        params = {
            "part": "snippet",
            "playlistId": playlist_id,
            "maxResults": 50,
            "key": api_key,
            "pageToken": next_page_token,
        }

        res = requests.get(url, params=params)
        res.raise_for_status()
        data = res.json()

        for item in data["items"]:
            snippet = item["snippet"]
            video_id = snippet["resourceId"]["videoId"]
            title = snippet["title"]
            video_url = f"https://www.youtube.com/watch?v={video_id}"
            videos.append((title, video_url))

        next_page_token = data.get("nextPageToken")
        if not next_page_token:
            break

    return videos


print(get_playlist_videos(API_KEY, playlist_id=PLAYLIST_ID))

[('【天文月報2025/06】XRISM News 3  Observation Results Part 2', 'https://www.youtube.com/watch?v=B4xVNXBZ__o'), ('【arXiv】アインシュタイン望遠鏡による中性子星の合体率の推定', 'https://www.youtube.com/watch?v=NOVe7S7TOYI')]


In [16]:
import json
from pathlib import Path


def load_sent_log(filepath: str) -> set[str]:
    path = Path(filepath)
    if not path.exists() or path.stat().st_size == 0:
        return set()
    with open(filepath, "r", encoding="utf-8") as f:
        return set(json.load(f))


def save_sent_log(sent_set: set[str], filepath: str):
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(list(sent_set), f, indent=2, ensure_ascii=False)


def get_next_unsent_video(
    all_videos: list[tuple[str, str]], sent_urls: set[str]
) -> tuple[str, str] | None:
    for title, url in all_videos:
        if url not in sent_urls:
            return (title, url)
    return None

In [17]:
all_videos = get_playlist_videos(API_KEY, PLAYLIST_ID)
sent_urls = load_sent_log("./sent_videos.json")

next_video = get_next_unsent_video(all_videos, sent_urls)
if next_video:
    title, url = next_video
    print(f"送信対象: {title} - {url}")

    # ↓ LINE送信処理をここに入れる

    sent_urls.add(url)
    save_sent_log(sent_urls, "sent_videos.json")
else:
    print("すべての動画はすでに送信済みです。")

送信対象: 【天文月報2025/06】XRISM News 3  Observation Results Part 2 - https://www.youtube.com/watch?v=B4xVNXBZ__o
